In [ ]:
### Data
import xarray as xr
from netCDF4 import Dataset
import numpy as np
from wrf import getvar, interplevel, smooth2d, destagger
from scipy.interpolate import griddata

### Plotting
import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
import matplotlib.ticker as mticker
import cartopy.crs as crs
from cartopy.feature import NaturalEarthFeature
import matplotlib.transforms as mtransforms

### Warnings
import warnings
warnings.filterwarnings('ignore')


In [ ]:
### COAMPS data

### Parameters
coamps_dir = '/p/work1/lloveras/nov2018/coamps_files/'
nx_c = 301
ny_c = 201
nz_c = 45

### Latitude and longitude
lats_c = np.fromfile(coamps_dir + 'latitu_sfc_000000_000000_1a0301x0201_2018111312_00000000_fcstfld', dtype='>f4')
lons_c = np.fromfile(coamps_dir + 'longit_sfc_000000_000000_1a0301x0201_2018111312_00000000_fcstfld', dtype='>f4')
lats_c = np.reshape(lats_c, [ny_c, nx_c])
lons_c = np.reshape(lons_c, [ny_c, nx_c])
lons_c[lons_c > 0] -= 360

### Pressure
pc_ctl = (np.fromfile(coamps_dir + 'ttlprs_sig_028485_000010_1a0301x0201_2018111312_00000000_fcstfld', dtype='>f4',count=nz_c*ny_c*nx_c))
pc_ctl = np.flip(np.reshape(pc_ctl, [nz_c, ny_c, nx_c]), axis=0)
pc_adj = (np.fromfile(coamps_dir + '9x/ttlprs_sig_028485_000010_1a0301x0201_2018111312_00000000_fcstfld', dtype='>f4',count=nz_c*ny_c*nx_c))
pc_adj = np.flip(np.reshape(pc_adj, [nz_c, ny_c, nx_c]), axis=0)
pp_c = pc_adj - pc_ctl

### Potential temperature
tc_ctl = (np.fromfile(coamps_dir + 'pottmp_sig_028485_000010_1a0301x0201_2018111312_00000000_fcstfld', dtype='>f4',count=nz_c*ny_c*nx_c))
tc_ctl = np.flip(np.reshape(tc_ctl, [nz_c, ny_c, nx_c]), axis=0)
tc_adj = (np.fromfile(coamps_dir + '9x/pottmp_sig_028485_000010_1a0301x0201_2018111312_00000000_fcstfld', dtype='>f4',count=nz_c*ny_c*nx_c))
tc_adj = np.flip(np.reshape(tc_adj, [nz_c, ny_c, nx_c]), axis=0)
t_c = tc_adj - tc_ctl

### Zonal wind
uc_ctl = (np.fromfile(coamps_dir + 'uuwind_sig_028485_000010_1a0301x0201_2018111312_00000000_fcstfld', dtype='>f4',count=nz_c*ny_c*nx_c))
uc_ctl = np.flip(np.reshape(uc_ctl, [nz_c, ny_c, nx_c]), axis=0)
uc_adj = (np.fromfile(coamps_dir + '9x/uuwind_sig_028485_000010_1a0301x0201_2018111312_00000000_fcstfld', dtype='>f4',count=nz_c*ny_c*nx_c))
uc_adj = np.flip(np.reshape(uc_adj, [nz_c, ny_c, nx_c]), axis=0)
u_c = uc_adj - uc_ctl

### Meridional wind
vc_ctl = (np.fromfile(coamps_dir + 'vvwind_sig_028485_000010_1a0301x0201_2018111312_00000000_fcstfld', dtype='>f4',count=nz_c*ny_c*nx_c))
vc_ctl = np.flip(np.reshape(vc_ctl, [nz_c, ny_c, nx_c]), axis=0)
vc_adj = (np.fromfile(coamps_dir + '9x/vvwind_sig_028485_000010_1a0301x0201_2018111312_00000000_fcstfld', dtype='>f4',count=nz_c*ny_c*nx_c))
vc_adj = np.flip(np.reshape(vc_adj, [nz_c, ny_c, nx_c]), axis=0)
v_c = vc_adj - vc_ctl

### Vertical wind
wc_ctl = (np.fromfile(coamps_dir + 'wwwind_sig_029735_000000_1a0301x0201_2018111312_00000000_fcstfld', dtype='>f4',count=(nz_c+1)*ny_c*nx_c))
wc_ctl = np.flip(np.reshape(wc_ctl, [nz_c+1, ny_c, nx_c]), axis=0)
wc_adj = (np.fromfile(coamps_dir + '9x/wwwind_sig_029735_000000_1a0301x0201_2018111312_00000000_fcstfld', dtype='>f4',count=(nz_c+1)*ny_c*nx_c))
wc_adj = np.flip(np.reshape(wc_adj, [nz_c+1, ny_c, nx_c]), axis=0)
wc_ctl = destagger(wc_ctl,0)
wc_adj = destagger(wc_adj,0)
w_c = wc_adj - wc_ctl

### Water vapor
qc_ctl = (np.fromfile(coamps_dir + 'wvapor_sig_028485_000010_1a0301x0201_2018111312_00000000_fcstfld', dtype='>f4',count=nz_c*ny_c*nx_c))
qc_ctl = np.flip(np.reshape(qc_ctl, [nz_c, ny_c, nx_c]), axis=0)
qc_adj = (np.fromfile(coamps_dir + '9x/wvapor_sig_028485_000010_1a0301x0201_2018111312_00000000_fcstfld', dtype='>f4',count=nz_c*ny_c*nx_c))
qc_adj = np.flip(np.reshape(qc_adj, [nz_c, ny_c, nx_c]), axis=0)
q_c = qc_adj - qc_ctl

In [ ]:
### WRF data
nx_w = 400
ny_w = 250
nz_w = 44
ctl_dir = '/p/work1/lloveras/nov2018/30km_files/ctl/wrfout_d01_2018-11-13_12_00_00'

### Open the netCDF file for reading
ncfile_ctl = Dataset(ctl_dir)

### Latitude and longitude
lats_w = np.asarray(getvar(ncfile_ctl,'lat',timeidx=0))
lons_w = np.asarray(getvar(ncfile_ctl,'lon',timeidx=0))
lons_w[lons_w > 0] -= 360

### Full pressure in hPa
p_w = np.asarray(getvar(ncfile_ctl,'pressure',timeidx=0))

In [ ]:
### Interpolate perturbations onto WRF grid

### Start by interpolating WRF pressure field onto COAMPS horizontal grid
p_w_cxy = np.zeros((nz_w,ny_c,nx_c))
for k in range(nz_w):
    p_w_cxy[k,:,:] = griddata((lons_w.ravel(), lats_w.ravel()),
                              p_w[k,:,:].ravel(), (lons_c, lats_c), method='linear')

pc = (pc_ctl + pc_adj)/2

### Now interpolate COAMPS perturbations onto WRF pressure levels
pp_c_wz = np.zeros([nz_w, ny_c, nx_c])
t_c_wz = np.zeros([nz_w, ny_c, nx_c])
u_c_wz = np.zeros((nz_w, ny_c, nx_c))
v_c_wz = np.zeros((nz_w, ny_c, nx_c))
w_c_wz = np.zeros((nz_w, ny_c, nx_c))
qv_c_wz = np.zeros((nz_w, ny_c, nx_c))
for k in range(nz_w):
    pp_c_wz[k,:,:] = np.nan_to_num(interplevel(pp_c, pc, p_w_cxy[k,:,:]))
    t_c_wz[k,:,:] = np.nan_to_num(interplevel(t_c, pc, p_w_cxy[k,:,:]))
    u_c_wz[k,:,:] = np.nan_to_num(interplevel(u_c, pc, p_w_cxy[k,:,:]))
    v_c_wz[k,:,:] = np.nan_to_num(interplevel(v_c, pc, p_w_cxy[k,:,:]))
    w_c_wz[k,:,:] = np.nan_to_num(interplevel(w_c, pc, p_w_cxy[k,:,:]))
    qv_c_wz[k,:,:] = np.nan_to_num(interplevel(q_c, pc, p_w_cxy[k,:,:]))

### Finally interpolate perturbations onto WRF horizontal grid
pp_c_w = np.zeros((nz_w, ny_w, nx_w))
t_c_w = np.zeros((nz_w, ny_w, nx_w))
u_c_w = np.zeros((nz_w, ny_w, nx_w))
v_c_w = np.zeros((nz_w, ny_w, nx_w))
w_c_w = np.zeros((nz_w, ny_w, nx_w))
qv_c_w = np.zeros((nz_w, ny_w, nx_w))
for k in range(nz_w):
    pp_c_w[k,:,:] = np.nan_to_num(griddata((lons_c.ravel(), lats_c.ravel()), 
                                          pp_c_wz[k,:,:].ravel(), (lons_w, lats_w), method='linear'))
    t_c_w[k,:,:] = np.nan_to_num(griddata((lons_c.ravel(), lats_c.ravel()), 
                                          t_c_wz[k,:,:].ravel(), (lons_w, lats_w), method='linear'))
    u_c_w[k,:,:] = np.nan_to_num(griddata((lons_c.ravel(), lats_c.ravel()), 
                                          u_c_wz[k,:,:].ravel(), (lons_w, lats_w), method='linear'))
    v_c_w[k,:,:] = np.nan_to_num(griddata((lons_c.ravel(), lats_c.ravel()), 
                                          v_c_wz[k,:,:].ravel(), (lons_w, lats_w), method='linear'))
    w_c_w[k,:,:] = np.nan_to_num(griddata((lons_c.ravel(), lats_c.ravel()), 
                                          w_c_wz[k,:,:].ravel(), (lons_w, lats_w), method='linear'))
    qv_c_w[k,:,:] = np.nan_to_num(griddata((lons_c.ravel(), lats_c.ravel()), 
                                          qv_c_wz[k,:,:].ravel(), (lons_w, lats_w), method='linear'))


In [ ]:
### Save the output
np.save('/p/work1/lloveras/nov2018/data_avail/lats',lats_w)
np.save('/p/work1/lloveras/nov2018/data_avail/lons',lons_w)
np.save('/p/work1/lloveras/nov2018/data_avail/p',p_w)
np.save('/p/work1/lloveras/nov2018/data_avail/dp',pp_c_w)
np.save('/p/work1/lloveras/nov2018/data_avail/dt',t_c_w)
np.save('/p/work1/lloveras/nov2018/data_avail/du',u_c_w)
np.save('/p/work1/lloveras/nov2018/data_avail/dv',v_c_w)
np.save('/p/work1/lloveras/nov2018/data_avail/dw',w_c_w)
np.save('/p/work1/lloveras/nov2018/data_avail/dqv',qv_c_w)

In [ ]:
test = np.load('/p/work1/lloveras/nov2018/data_avail_nov2018/dqv.npy')

In [ ]:
plt.contourf(test[15,:,:]*1000.,levels=np.arange(0,2,0.2),extend='max');plt.colorbar()

In [ ]:
np.amax(np.abs(test[30,:,:]*1000.))